In [575]:
# Курсовой проект для курса "Python для Data Science"

# Материалы к проекту (файлы):
# train.csv
# test.csv

# Задание:
# Используя данные из train.csv, построить
# модель для предсказания цен на недвижимость (квартиры).
# С помощью полученной модели предсказать
# цены для квартир из файла test.csv.

# Целевая переменная:
# Price

# Основная метрика:
# R2 - коэффициент детерминации (sklearn.metrics.r2_score)

# Вспомогательная метрика:
# MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error)

In [576]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [577]:
data = pd.read_csv('train.csv')

In [578]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [579]:
data['LifeSquare'] = data['LifeSquare'].fillna(data['LifeSquare'].mean())
data['Healthcare_1'] = data['Healthcare_1'].fillna(data['Healthcare_1'].mean())

In [580]:
data.loc[data['Rooms'] > 5]
data.loc[377, 'Rooms'] = 3
data.loc[1454, 'Rooms'] = 2
data.loc[2170, 'Rooms'] = 3
data.loc[8849, 'Rooms'] = 3

In [581]:
del data['HouseFloor']

In [582]:
data.loc[data['LifeSquare'] > data['Square'], 'Square'] = data['LifeSquare']

In [583]:
data.loc[(data['KitchenSquare'] == 0) & (data['Square'] > data['LifeSquare']), 'KitchenSquare'] = \
data['Square'] - data['LifeSquare']

In [584]:
data.loc[(data['Rooms'] < 1) & (data['LifeSquare'] > 35), 'Rooms'] = 3

In [585]:
data.loc[(data['Rooms'] < 1)] = 1

In [586]:
data.loc[(data['Square'] < 15) & (data['LifeSquare'] < 15) & (data['Rooms'] <= 1), 'Square'] = \
data.loc[data['Rooms'] <= 1, 'Square'].mean()

In [587]:
data.loc[(data['Square'] < 15) & (data['LifeSquare'] < 15) & (data['Rooms'] == 3), 'Square'] = \
data.loc[data['Rooms'] == 3, 'Square'].mean()

In [588]:
data.loc[(data['Rooms'] == 3) & (data['LifeSquare'] < 35), 'LifeSquare'] = \
data.loc[data['Rooms'] == 3, 'LifeSquare'].mean()

In [589]:
data.loc[(data['Rooms'] == 2) & (data['LifeSquare'] < 20), 'LifeSquare'] = \
data.loc[data['Rooms'] == 2, 'LifeSquare'].mean()

In [590]:
data.loc[(data['Rooms'] == 1) & (data['LifeSquare'] < 15), 'LifeSquare'] = \
data.loc[data['Rooms'] == 1, 'LifeSquare'].mean()

In [591]:
#data.loc[(data['Price'] > data['Price'].mean()) & (data['Rooms'] == 1), 'Price'] = \
#data.loc[data['Rooms'] == 1, 'Price'].mean()

In [592]:
#data.loc[(data['Price'] > data['Price'].mean()) & (data['Rooms'] == 2), 'Price'] = \
#data.loc[data['Rooms'] == 2, 'Price'].mean()

In [593]:
#data.loc[(data['Price'] > data['Price'].mean()) & (data['Rooms'] == 3), 'Price'] = \
#data.loc[data['Rooms'] == 3, 'Price'].mean()

In [594]:
data.drop(4328, inplace=True)

In [595]:
data.loc[(data['Rooms'] == 4) & (data['LifeSquare'] < 45), 'LifeSquare'] = \
data.loc[data['Rooms'] == 4, 'LifeSquare'].mean()
data.loc[(data['Rooms'] == 5) & (data['LifeSquare'] < 55), 'LifeSquare'] = \
data.loc[data['Rooms'] == 5, 'LifeSquare'].mean()

In [596]:
data.loc[(data['KitchenSquare'] > data['LifeSquare']) 
         & (data['KitchenSquare'] + data['LifeSquare'] == data['Square']), ['LifeSquare', 'KitchenSquare']] = \
data.loc[(data['KitchenSquare'] > data['LifeSquare']) 
         & (data['KitchenSquare'] + data['LifeSquare'] == data['Square']), ['KitchenSquare', 'LifeSquare']].values

In [597]:
data.loc[(data['KitchenSquare'] < 10) & (data['Rooms'] == 3), 'KitchenSquare'] =\
data.loc[data['Rooms'] == 3, 'KitchenSquare'].mean()

data.loc[(data['KitchenSquare'] < 7) & (data['Rooms'] == 2), 'KitchenSquare'] =\
data.loc[data['Rooms'] == 2, 'KitchenSquare'].mean()

data.loc[(data['KitchenSquare'] < 5) & (data['Rooms'] == 1), 'KitchenSquare'] =\
data.loc[data['Rooms'] == 1, 'KitchenSquare'].mean()

In [598]:
data['Square_2'] = data['Square'] ** 2

In [599]:
data.loc[data['Square'] > 500, 'Square'] = data.loc[data['Rooms'] == 2, 'Square'].mean()

In [600]:
data.loc[data['KitchenSquare'] > 500, 'KitchenSquare'] = data.loc[data['Rooms'] == 1, 'KitchenSquare'].mean()

In [601]:
data.loc[data['Square'] < data['LifeSquare'], ['Square', 'LifeSquare']] = \
data.loc[data['Square'] < data['LifeSquare'], ['LifeSquare', 'Square']]

In [602]:
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 4), 'KitchenSquare'] = \
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 4), 'KitchenSquare'].mean()

In [603]:
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 5), 'KitchenSquare'] = \
data.loc[(data['KitchenSquare'] < 20) & (data['Rooms'] == 5), 'KitchenSquare'].mean()

In [604]:
data = pd.get_dummies(data)

In [605]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [606]:
train.shape, valid.shape

((6999, 26), (3000, 26))

In [607]:
district_stat = train.groupby(['DistrictId', 'Rooms'])[['Price']].mean().reset_index().rename(columns={'Price' : 'mean_price'})
district_stat.shape

(623, 3)

In [608]:
train = pd.merge(train, district_stat, on=['DistrictId', 'Rooms'], how='left')
train['mean_price'].isnull().sum()

0

In [609]:
valid = pd.merge(valid, district_stat, on=['DistrictId', 'Rooms'], how='left')
valid['mean_price'].isnull().sum()

51

In [610]:
rooms_stat = train.groupby(['Rooms'])[['Price']].mean().reset_index().rename(columns={'Price':'mean_price2'})

In [611]:
train = pd.merge(train, rooms_stat, on=['Rooms'], how='left')

In [612]:
valid = pd.merge(valid, rooms_stat, on=['Rooms'], how='left')

In [613]:
valid['mean_price2'].isnull().sum()

0

In [614]:
valid['mean_price'].fillna(valid['mean_price2'], inplace=True)

In [615]:
# линейная регрессия

In [616]:
lr = LinearRegression()
features_for_predict = ['Square', 'Square_2','LifeSquare', 'Rooms', 
       'Floor', 'HouseYear', 'Ecology_1',
        'Healthcare_1',
       'Social_1', 'Social_2', 'Ecology_2_A', 'Ecology_2_B',
        'Ecology_3_A', 'Ecology_3_B',
       'Shops_1', 'Shops_2_A',
       'Shops_2_B', 'mean_price',
      ]


In [617]:
lr.fit(train.loc[:, features_for_predict], train['Price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [618]:
pred_train = lr.predict(train.loc[:, features_for_predict])

In [619]:
r2_score(train['Price'], pred_train)

0.7318724017829429

In [620]:
pred = lr.predict(valid.loc[:, features_for_predict])

In [621]:
valid_pred = pd.DataFrame({'valid_price' : valid['Price'], 'pred_valid_price' : pred})
valid_pred.head(10)

,valid_price,pred_valid_price
0,187042.020422,209174.019169
1,258182.675237,200146.517738
2,219945.304640,215058.525468
3,102666.191762,141694.463286
4,158287.271791,151363.261029
5,102577.541221,216901.996533
6,377445.078933,351774.111838
7,260094.938025,222797.791590
8,148458.941020,146956.092991
9,184036.005146,199589.466233


In [622]:
r2_score(valid['Price'], pred)

0.5725824608160162

#### RF

In [623]:
# random forest
from sklearn.ensemble import RandomForestRegressor as RF

In [624]:
random_state = 42

In [625]:
rf = RF(max_depth=8, n_estimators=49, random_state=42)

In [626]:
rf.fit(train.loc[:, features_for_predict], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=49, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [627]:
pred = rf.predict(train.loc[:, features_for_predict])

In [628]:
r2_score(train['Price'], pred)

0.84130355926252

In [629]:
pred_valid = rf.predict(valid.loc[:, features_for_predict])

In [630]:
r2_score(valid['Price'], pred_valid)

0.6436349900767306

In [631]:
# for i in range(1, 100, 10):
#     print(i)
#     rf = RF(max_depth=8, n_estimators=20, min_samples_leaf=i)
#     rf.fit(train.loc[:, features_for_predict], train['Price'])
#     pred = rf.predict(train.loc[:, features_for_predict])
#     print(r2_score(train['Price'], pred))
#     pred_valid = rf.predict(valid.loc[:, features_for_predict])
#     print(r2_score(valid['Price'], pred_valid))
#     print()

### предсказание на тесте

In [632]:
test = pd.read_csv('test.csv')

In [633]:
test['LifeSquare'] = test['LifeSquare'].fillna(test['LifeSquare'].mean())
test['Healthcare_1'] = test['Healthcare_1'].fillna(test['Healthcare_1'].mean())

In [634]:
del test['HouseFloor']

In [635]:
test.loc[test['LifeSquare'] > test['Square'], 'Square'] = test['LifeSquare']

In [636]:
test.loc[(test['KitchenSquare'] == 0) & (test['Square'] > test['LifeSquare']), 'KitchenSquare'] = \
test['Square'] - test['LifeSquare']

In [637]:
test.loc[(test['Rooms'] < 1) & (test['LifeSquare'] > 35), 'Rooms'] = 3

In [638]:
test.loc[(test['Rooms'] < 1)] = 1

In [639]:
test.loc[(test['Square'] < 15) & (test['LifeSquare'] < 15) & (test['Rooms'] <= 1), 'Square'] = \
test.loc[test['Rooms'] <= 1, 'Square'].mean()

In [640]:
test.loc[(test['Square'] < 15) & (test['LifeSquare'] < 15) & (test['Rooms'] == 3), 'Square'] = \
test.loc[test['Rooms'] == 3, 'Square'].mean()

In [641]:
test.loc[(test['Rooms'] == 3) & (test['LifeSquare'] < 35), 'LifeSquare'] = \
test.loc[test['Rooms'] == 3, 'LifeSquare'].mean()

In [642]:
test.loc[(test['Rooms'] == 2) & (test['LifeSquare'] < 20), 'LifeSquare'] = \
test.loc[test['Rooms'] == 2, 'LifeSquare'].mean()

In [643]:
test.loc[(test['Rooms'] == 1) & (test['LifeSquare'] < 15), 'LifeSquare'] = \
test.loc[test['Rooms'] == 1, 'LifeSquare'].mean()

In [644]:
test.loc[(test['Rooms'] == 4) & (test['LifeSquare'] < 45), 'LifeSquare'] = \
test.loc[test['Rooms'] == 4, 'LifeSquare'].mean()
test.loc[(test['Rooms'] == 5) & (test['LifeSquare'] < 55), 'LifeSquare'] = \
test.loc[test['Rooms'] == 5, 'LifeSquare'].mean()

In [645]:
test.loc[(test['KitchenSquare'] > test['LifeSquare']) 
         & (test['KitchenSquare'] + test['LifeSquare'] == test['Square']), ['LifeSquare', 'KitchenSquare']] = \
test.loc[(test['KitchenSquare'] > test['LifeSquare']) 
         & (test['KitchenSquare'] + test['LifeSquare'] == test['Square']), ['KitchenSquare', 'LifeSquare']].values

In [646]:
test.loc[(test['KitchenSquare'] < 10) & (test['Rooms'] == 3), 'KitchenSquare'] =\
test.loc[test['Rooms'] == 3, 'KitchenSquare'].mean()

test.loc[(test['KitchenSquare'] < 7) & (test['Rooms'] == 2), 'KitchenSquare'] =\
test.loc[test['Rooms'] == 2, 'KitchenSquare'].mean()

test.loc[(test['KitchenSquare'] < 5) & (test['Rooms'] == 1), 'KitchenSquare'] =\
test.loc[test['Rooms'] == 1, 'KitchenSquare'].mean()

In [647]:
test['Square_2'] = test['Square'] ** 2

In [648]:
test.loc[test['Square'] > 500, 'Square'] = test.loc[test['Rooms'] == 2, 'Square'].mean()

In [649]:
test.loc[test['KitchenSquare'] > 500, 'KitchenSquare'] = test.loc[test['Rooms'] == 1, 'KitchenSquare'].mean()

In [650]:
test.loc[test['Square'] < test['LifeSquare'], ['Square', 'LifeSquare']] = \
test.loc[test['Square'] < test['LifeSquare'], ['LifeSquare', 'Square']]

In [651]:
test.loc[(test['KitchenSquare'] < 20) & (test['Rooms'] == 4), 'KitchenSquare'] = \
test.loc[(test['KitchenSquare'] < 20) & (test['Rooms'] == 4), 'KitchenSquare'].mean()

In [652]:
test.loc[(test['KitchenSquare'] < 20) & (test['Rooms'] == 5), 'KitchenSquare'] = \
test.loc[(test['KitchenSquare'] < 20) & (test['Rooms'] == 5), 'KitchenSquare'].mean()

In [653]:
test = pd.merge(test, district_stat, on=['DistrictId', 'Rooms'], how='left')

In [654]:
test = pd.merge(test, rooms_stat, on=['Rooms'], how='left')

In [655]:
test = pd.get_dummies(test)

In [656]:
test['mean_price'].isnull().sum()

98

In [657]:
test['mean_price'].fillna(test['mean_price2'].mean(), inplace=True)

In [658]:
test['mean_price2'].isnull().sum()

3

In [659]:
test['mean_price2'].fillna(test['mean_price2'].mean(), inplace=True)

In [660]:
test_pred = rf.predict(test.loc[:, features_for_predict])

In [662]:
test['Price'] = test_pred

In [663]:
test.loc[:, ['Id', 'Price']].to_csv('VBurtcev_predictions.csv')